In [2]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import pandas as pd
import trino
import io
import zipfile
import anytree
from typing import Dict, List, Union
import json

In [3]:
dotenv_dir = os.environ.get(
    "CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src")
)

dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [4]:
# creates s3 resource for DEBUGGING purposes to access S3
s3_resource_osc = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ["S3_DEV_ENDPOINT"],
    aws_access_key_id=os.environ["S3_DEV_ACCESS_KEY"],
    aws_secret_access_key=os.environ["S3_DEV_SECRET_KEY"],
)

In [5]:
# creating trino connection
import trino
from sqlalchemy.engine import create_engine

sqlstring = "trino://{user}@{host}:{port}/".format(
    user=os.environ["TRINO_USER"],
    host=os.environ["TRINO_HOST"],
    port=os.environ["TRINO_PORT"],
)
sqlargs = {
    "auth": trino.auth.JWTAuthentication(os.environ["TRINO_PASSWD"]),
    "http_scheme": "https",
}
engine = create_engine(sqlstring, connect_args=sqlargs)
print("connecting with engine " + str(engine))
connection = engine.connect()

connecting with engine Engine(trino://DavWinkel@trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org:443/)


In [12]:
def calculate_matches_list_to_dataframe(
    input_list: List[str],
    tmp_df: pd.DataFrame,
    key_column: str,
    relevant_output_column: str,
) -> List[str]:

    # reduces search universe by only inlcuding the matches
    filtered_df = tmp_df[tmp_df[key_column].isin(input_list)]

    if not filtered_df[key_column].is_unique:
        warnings.warn(
            f"Output column {key_column} is not unique and thus there can "
            f"not be a 1:1 mapping ensured"
        )

    tmp_df_join = pd.DataFrame(input_list, columns=[f"{key_column}_left"])
    tmp_merged = pd.merge(
        left=tmp_df_join,
        right=filtered_df,
        left_on=f"{key_column}_left",
        right_on=key_column,
        how="left",
    )

    # converting NaNs to None
    tmp_merged = tmp_merged.where(pd.notnull(tmp_merged), None)

    return tmp_merged[relevant_output_column].tolist()


def identify_direct_issuer_lei_by_isin(
    list_isin: List[str], time_stamp_str: str, s3_resource, trino_connection
) -> List[str]:

    if trino_connection is not None:
        TABLENAME_ISIN_LEI = "gleif_isin_lei"
        sqlquery = f"select * from osc_datacommons_dev.gleif.{TABLENAME_ISIN_LEI}"
        tmp_df = pd.read_sql(sqlquery, engine).convert_dtypes()

    elif s3_resource is not None:
        files = []
        for file in s3_resource.Bucket(os.environ["S3_DEV_BUCKET"]).objects.filter(
            Prefix="trino/gleif/"
        ):
            if file.key.endswith("parquet"):
                if "ISIN_LEI_mapping" in file.key:
                    files.append(file.key)

        buffer = io.BytesIO()
        parquet_file = s3_resource.Object(os.environ["S3_DEV_BUCKET"], files[-1])
        parquet_file.download_fileobj(buffer)
        tmp_df = pd.read_parquet(buffer)
    else:
        raise ValueError("Provide either a trino or a S3 data source connection")

    return calculate_matches_list_to_dataframe(
        input_list=list_isin,
        tmp_df=tmp_df,
        key_column="isin",
        relevant_output_column="lei",
    )


def identify_ultimate_issuer_lei_by_direct_issuer_lei(
    list_direct_issuer_LEI: List[str],
    time_stamp_str: str,
    s3_resource,
    trino_connection,
) -> List[str]:
    if trino_connection is not None:
        TABLENAME_DIRECT_ULTIMATE_ISSUER = "gleif_direct_issuer_ultimate_issuer"
        sqlquery = f"select * from osc_datacommons_dev.gleif.{TABLENAME_DIRECT_ULTIMATE_ISSUER}"
        tmp_df = pd.read_sql(sqlquery, engine).convert_dtypes()

    elif s3_resource is not None:
        files = []
        for file in s3_resource.Bucket(os.environ["S3_DEV_BUCKET"]).objects.filter(
            Prefix="trino/gleif/"
        ):
            if file.key.endswith("parquet"):
                if "ultimate_issuer_LEI_mapping" in file.key:
                    files.append(file.key)

        buffer = io.BytesIO()
        parquet_file = s3_resource.Object(os.environ["S3_DEV_BUCKET"], files[-1])
        parquet_file.download_fileobj(buffer)
        tmp_df = pd.read_parquet(buffer)
    else:
        raise ValueError("Provide either a trino or a S3 data source connection")

    # direct issuer == ultimate issuer
    list_relevant_ultimate_issuers = tmp_df[
        tmp_df["ultimate_parent_issuer_lei"].isin(list_direct_issuer_LEI)
    ]["ultimate_parent_issuer_lei"].tolist()
    list_already_ultimate_issuer_matches = [
        direct_issuer if direct_issuer in list_relevant_ultimate_issuers else None
        for direct_issuer in list_direct_issuer_LEI
    ]
    # direct issuer != ultimate issuer
    list_direct_issuer_to_ultimate_issuer_matches = calculate_matches_list_to_dataframe(
        input_list=list_direct_issuer_LEI,
        tmp_df=tmp_df,
        key_column="direct_issuer_lei",
        relevant_output_column="ultimate_parent_issuer_lei",
    )

    list_merged_ultimate_issuer = [
        entry_tuple[0] if entry_tuple[0] is not None else entry_tuple[1]
        for entry_tuple in zip(
            list_already_ultimate_issuer_matches,
            list_direct_issuer_to_ultimate_issuer_matches,
        )
    ]

    return list_merged_ultimate_issuer

In [13]:
def identify_company_hierarchy_by_ultimate_isser_lei(
    list_ultimate_issuer_LEI: List[str], time_stamp_str: str, s3_resource
) -> List[Dict]:

    files = []
    if s3_resource is not None:
        for file in s3_resource.Bucket(os.environ["S3_DEV_BUCKET"]).objects.filter(
            Prefix="trino/gleif/"
        ):
            if file.key.endswith("json"):
                if "company_hierarchy_tree" in file.key:
                    files.append(file.key)

        json_object = s3_resource.Object(os.environ["S3_DEV_BUCKET"], files[-1])
        json_obj_dec = json_object.get()["Body"].read().decode("utf-8")
        company_hierarchy_tree = json.loads(json_obj_dec)
    else:
        raise ValueError("Provide a S3 data source connection")

    dict_lookup_ultimate_issuer_index = dict(
        [
            (value.get("entity_LEI"), index)
            for index, value in enumerate(company_hierarchy_tree)
        ]
    )

    tmp_list_company_hierarchy = []
    for ultimate_issuer_LEI in list_ultimate_issuer_LEI:
        if ultimate_issuer_LEI in dict_lookup_ultimate_issuer_index.keys():
            index_ultimate_issuer_LEI = dict_lookup_ultimate_issuer_index.get(
                ultimate_issuer_LEI
            )
            tmp_list_company_hierarchy.append(
                company_hierarchy_tree[index_ultimate_issuer_LEI]
            )
        else:
            tmp_list_company_hierarchy.append(None)

    return tmp_list_company_hierarchy


def identify_company_hierarchy_path_direct_issuer_to_ultimate_issuer(
    list_direct_issuer_LEI: List[str], list_dict_company_hierarchy: List[Dict]
) -> List:

    list_company_path = []
    for pair in zip(list_direct_issuer_LEI, list_dict_company_hierarchy):
        direct_issuer = pair[0]
        dict_company_hierarchy = pair[1]
        if direct_issuer is None or dict_company_hierarchy is None:
            list_company_path.append(None)
        else:
            tmp_company_path = find_company_hierarchy_path(
                target_LEI_to_find=direct_issuer,
                sub_dict_to_check=dict_company_hierarchy,
            )
            list_company_path.append(tmp_company_path)
    return list_company_path


def find_company_hierarchy_path(
    target_LEI_to_find: str, sub_dict_to_check: Dict, string_encoded_path=None
) -> Union[List, None]:
    """
    Recursively loops through the sub_dict_to_check to find the target_LEI_to_find value while storing
     the path passing though the dictionary. Returns a list of LEIs, starting from the ultimate issuer and
     going through all legal entities between the direct issuer and the ultimate issuer until it finally reaches the
     direct issuer which is the last element of the list.
     In the case that direct issuer = ultimate issuer a list with just one LEI element will be returned.
    :param target_LEI_to_find:
    :param sub_dict_to_check:
    :param string_encoded_path:
    :return:
    """

    if string_encoded_path is None:  # only for initial run
        string_encoded_path = sub_dict_to_check.get("entity_LEI")

    if target_LEI_to_find == sub_dict_to_check.get("entity_LEI"):
        tmp_list = string_encoded_path.split("_")
        return tmp_list
    else:
        if "entity_children" in sub_dict_to_check:
            for dict_entry in sub_dict_to_check.get("entity_children"):
                tmp_string_encoded_path = (
                    f'{string_encoded_path}_{dict_entry.get("entity_LEI")}'
                )
                tmp_return = find_company_hierarchy_path(
                    target_LEI_to_find=target_LEI_to_find,
                    sub_dict_to_check=dict_entry,
                    string_encoded_path=tmp_string_encoded_path,
                )
                if tmp_return is None:
                    pass
                else:
                    return tmp_return
        else:
            return None

In [14]:
def retrieve_company_info_by_lei(
    list_lei: List[str],
    s3_resource,
    trino_connection=None,
    time_stamp_str: str = None,
    include_company_hierarchy_tree: bool = False,
    include_company_path: bool = False,
):

    list_ultimate_issuer_lei = identify_ultimate_issuer_lei_by_direct_issuer_lei(
        list_direct_issuer_LEI=list_lei,
        time_stamp_str=time_stamp_str,
        s3_resource=s3_resource,
        trino_connection=trino_connection,
    )
    list_company_hierarchy = None
    list_company_hierarchy_path = None
    if include_company_hierarchy_tree or include_company_path:
        list_company_hierarchy = identify_company_hierarchy_by_ultimate_isser_lei(
            list_ultimate_issuer_LEI=list_ultimate_issuer_lei,
            time_stamp_str=time_stamp_str,
            s3_resource=s3_resource,
        )
        if include_company_path:
            list_company_hierarchy_path = (
                identify_company_hierarchy_path_direct_issuer_to_ultimate_issuer(
                    list_direct_issuer_LEI=list_lei,
                    list_dict_company_hierarchy=list_company_hierarchy,
                )
            )
        if not include_company_hierarchy_tree:
            list_company_hierarchy = None

    return dict(
        {
            "list_ultimate_parent_LEI": list_ultimate_issuer_lei,
            "list_company_hierarchy": list_company_hierarchy,
            "list_company_hierarchy_path": list_company_hierarchy_path,
        }
    )


def retrieve_company_info_by_isin(
    list_isin: List[str],
    s3_resource,
    trino_connection=None,
    time_stamp_str: str = None,
    include_company_hierarchy_tree: bool = False,
    include_company_path: bool = False,
):

    list_direct_issuer_lei = identify_direct_issuer_lei_by_isin(
        list_isin=list_isin,
        time_stamp_str=time_stamp_str,
        s3_resource=s3_resource,
        trino_connection=trino_connection,
    )
    tmp_comp_dict = retrieve_company_info_by_lei(
        list_lei=list_direct_issuer_lei,
        s3_resource=s3_resource,
        trino_connection=trino_connection,
        time_stamp_str=time_stamp_str,
        include_company_hierarchy_tree=include_company_hierarchy_tree,
        include_company_path=include_company_path,
    )
    tmp_dict = {"list_direct_parent_LEI": list_direct_issuer_lei}
    tmp_dict.update(tmp_comp_dict)
    return tmp_dict

# Examples for the service functions:

In [15]:
# TODO Passing a trino connection is considerably slower than just pulling data from the S3 bucket (by setting
# trino_connection=None). Speed should increase by doing the filtering already by an SQL statement on the DB
# instead of fetching fetching the entire dataset and do the filtering via pandas

list_isin = ["US38141GXK56", "US05600LAC00"]
result = retrieve_company_info_by_isin(
    list_isin=list_isin,
    s3_resource=s3_resource_osc,
    trino_connection=None, # connection,
    time_stamp_str=None,
    include_company_hierarchy_tree=False,
    include_company_path=True,
)

print(result)

list_lei = ["529900HLUAHG5YJSGB42", "549300JD0S5IZJE9LY15"]
result = retrieve_company_info_by_lei(
    list_lei=list_lei,
    s3_resource=s3_resource_osc,
    trino_connection=None, # connection,
    time_stamp_str=None,
    include_company_hierarchy_tree=False,
    include_company_path=True,
)
print(result)

{'list_direct_parent_LEI': ['784F5XWPLTWKTBV3E584', '5299006ZHG3IXU0PNJ56'], 'list_ultimate_parent_LEI': ['784F5XWPLTWKTBV3E584', 'YEH5ZCD6E441RHVHD759'], 'list_company_hierarchy': None, 'list_company_hierarchy_path': [['784F5XWPLTWKTBV3E584'], ['YEH5ZCD6E441RHVHD759', '5299006ZHG3IXU0PNJ56']]}
{'list_ultimate_parent_LEI': ['529900K9B0N5BT694847', '5493000C01ZX7D35SD85'], 'list_company_hierarchy': None, 'list_company_hierarchy_path': [['529900K9B0N5BT694847', '529900HLUAHG5YJSGB42'], ['5493000C01ZX7D35SD85', '549300JD0S5IZJE9LY15']]}
